<a href="https://colab.research.google.com/github/na-learning/ML/blob/main/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

Load the NYC taxi dataset into a Pandas DataFrame and do a few basic checks to ensure the data is loaded properly. Note, there are several months of data that can be used. For simplicity, use the Yellow Taxi 2022-01 parquet file here. Here are your tasks:

1. Load the yellow_tripdata_2022-01.parquet file into Pandas.
2. Print the first 5 rows of data.
3. Drop any rows of data that contain NULL values.
4. Create a new feature, 'trip_duration' that captures the duration of the trip in minutes.
5. Create a varible named 'target_variable' to store the name of the thing we're trying to predict, 'total_amount'.
6. Create a list called 'feature_cols' containing the feature names that we'll be using to predict our target variable. The list should contain 'VendorID', 'trip_distance', 'payment_type', 'PULocationID', 'DOLocationID', and 'trip_duration'.

In [14]:
# Load the dataset into a pandas DataFrame (from https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)
data = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet', engine='pyarrow')

In [15]:
# Display the first few rows of the dataset
print(data.shape)
print(data.head(5))

(2463931, 19)
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2022-01-01 00:35:40   2022-01-01 00:53:29              2.0   
1         1  2022-01-01 00:33:43   2022-01-01 00:42:07              1.0   
2         2  2022-01-01 00:53:21   2022-01-01 01:02:19              1.0   
3         2  2022-01-01 00:25:21   2022-01-01 00:35:23              1.0   
4         2  2022-01-01 00:36:48   2022-01-01 01:14:20              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           3.80         1.0                  N           142           236   
1           2.10         1.0                  N           236            42   
2           0.97         1.0                  N           166           166   
3           1.09         1.0                  N           114            68   
4           4.30         1.0                  N            68           163   

   payment_type  fare_amount  extra  mta_tax  tip_amount  to

In [16]:
# Drop rows with missing values.
data = data.dropna(axis=0)
print(data.shape)

(2392428, 19)


In [17]:
# Create new feature, 'trip_duration'.
col = data.apply(lambda row: ((row.tpep_dropoff_datetime - row.tpep_pickup_datetime).seconds)/60, axis=1)
data = data.assign(trip_duration=col.values)

In [18]:
target_variable = "total_amount"

In [19]:
# Create a list called feature_col to store column names
feature_cols = ['VendorID', 'trip_distance', 'payment_type', 'PULocationID', 'DOLocationID', 'trip_duration']

Use Scikit-Learn's train_test_split to split the data into training and test sets. Don't forget to set the random state.

In [20]:
# Split dataset into training and test sets
# cols = ",".join(str(feature) for feature in feature_cols)
# cols += "," + str(target_variable)
# data = data[cols.split(",")]
X = data[feature_cols]
y = data[target_variable]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Create a model that always predicts the mean total fare of the training dataset. Use Scikit-Learn's mean_absolute_error to evaluate this model. Is it any good?

In [ ]:
# Create a baseline for mean absolute error of total amount
from sklearn.dummy import DummyRegressor
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(X_train, y_train)
y_pred = dummy_regr.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(mae)


9.198227928516678


1. Use Scikit-Learn's ColumnTransformer to preprocess the categorical and continuous features independently. Apply the StandardScaler to the continuous columns and OneHotEncoder to the categorical columns.

2. Integrate the preprocessor in the previous step with Scikit-Learn's LinearRegression model using a Pipeline.

3. Train the pipeline on the training data.

4. Evaluate the model using mean absolute error as a metric on the test data. Does the model beat the baseline?

In [ ]:
# Use Scikit-Learn's ColumnTransformer to preprocess the categorical and
# continuous features independently.
numerical_x = X.select_dtypes(include=['int64', 'float64']).columns
categorical_x = X.select_dtypes(include=['object', 'bool']).columns

print(numerical_x)
print(categorical_x)


t = [('num', StandardScaler(), numerical_x), ('cat', OneHotEncoder(), categorical_x)]

transformer = ColumnTransformer(transformers=t, remainder='passthrough')

Index(['VendorID', 'trip_distance', 'payment_type', 'PULocationID',
       'DOLocationID', 'trip_duration'],
      dtype='object')
Index([], dtype='object')


In [ ]:
# Create a pipeline object containing the column transformations and regression
# model.
pipeline = Pipeline(steps=[('trans', transformer), ('model', LinearRegression())])

In [ ]:
# Fit the pipeline on the training data.
pipeline.fit(X_train, y_train)

Pipeline(steps=[('trans',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  Index(['VendorID', 'trip_distance', 'payment_type', 'PULocationID',
       'DOLocationID', 'trip_duration'],
      dtype='object')),
                                                 ('cat', OneHotEncoder(),
                                                  Index([], dtype='object'))])),
                ('model', LinearRegression())])

In [ ]:
# Make predictions on the test data.
y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(mae)

2.9656198158340654


This model definitely beats the baseline model

1. Build a Random Forest Regressor model using Scikit-Learn's RandomForestRegressor and train it on the train data.

2. Evaluate the performance of the model on the test data using mean absolute error as a metric. Mess around with various input parameter configurations to see how they affect the model. Can you beat the performance of the linear regression model?

In [ ]:
# Build random forest regressor model
# rfr = RandomForestRegressor(n_estimators=100, random_state=42)
# Crashed with message that RAM used up
# rfr = RandomForestRegressor(n_estimators=50, n_jobs=10, random_state=42)
# Crashed with message that RAM used up

# Tweaking criterion
# Using criterion="absolute_error" crashed
# Using criterion="poisson" failed as negative values are no allowed for y

criterion = ["squared_error", "friedman_mse"]
for criteria in criterion:
  rfr = RandomForestRegressor(n_estimators=20, criterion=criteria, n_jobs=5, random_state=42)

  rfr.fit(X_train, y_train)
  y_pred = rfr.predict(X_test)
  print(f"Mean absolute error with criterion = '{criteria}':", mean_absolute_error(y_test, y_pred))

# MAE with criterion = 'squared_error' was better performing compared to 'friedman_mse' by a very minute margin
# criterion = 'squared_error' was also faster model

Mean absolute error with criterion = 'squared_error': 1.6975470941748965
Mean absolute error with criterion = 'friedman_mse': 1.6975775863504312


In [ ]:
# Tweaking n_estimators
for i in range(1,18,4):
  rfr = RandomForestRegressor(n_estimators=i, criterion='squared_error', n_jobs=5, random_state=42)

  rfr.fit(X_train, y_train)
  y_pred = rfr.predict(X_test)
  print(f"Mean absolute error with n_estimators = {i}:", mean_absolute_error(y_test, y_pred))

# MAE reduces as n_estimators increases except with n_estimators=1 and n_estimators=5 acting as an anamoly

Mean absolute error with n_estimators = 1: 2.0465458609577576
Mean absolute error with n_estimators = 5: 2.1893195700305217
Mean absolute error with n_estimators = 9: 1.899799678949222
Mean absolute error with n_estimators = 13: 1.7878418327949885
Mean absolute error with n_estimators = 17: 1.7282350195598852


In [ ]:
# Tweaking n_jobs
for i in range(1,12,2):
  rfr = RandomForestRegressor(n_estimators=20, criterion='squared_error', n_jobs=i, random_state=42)

  rfr.fit(X_train, y_train)
  y_pred = rfr.predict(X_test)
  print(f"Mean absolute error with n_jobs = {i}:", mean_absolute_error(y_test, y_pred))

# No difference in MAE between 1 - 11 n_jobs

Mean absolute error with n_jobs = 1: 1.6975470941748965
Mean absolute error with n_jobs = 3: 1.6975470941748965
Mean absolute error with n_jobs = 5: 1.6975470941748965
Mean absolute error with n_jobs = 7: 1.6975470941748965
Mean absolute error with n_jobs = 9: 1.6975470941748965
Mean absolute error with n_jobs = 11: 1.6975470941748965


1. Perform a grid-search on a Random Forest Regressor model. Only search the space for the parameters 'n_estimators', 'max_depth', and 'min_samples_split'. Note, this can take some time to run. Make sure you set reasonable boundaries for the search space. Use Scikit-Learn's GridSearchCV method.

2. After you've identified the best parameters, train a random forest regression model using these parameters on the full training data.

3. Evaluate the model from the previous step using the test data. How does your model perform?

In [21]:
# Create a Random Forest Regressor
rfr = RandomForestRegressor()

# Define the hyperparameters to tune.
param_grid = {'n_estimators': [5, 10, 15, 20], 'max_depth': [2, 4, 8, 15], 'min_samples_split': [2, 4, 8]}

# Create a GridSearchCV object
grid_rf_class = GridSearchCV(
    estimator=rfr,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    cv=5,
    refit=True, return_train_score=True)
print(grid_rf_class)

grid_rf_class.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [2, 4, 8, 15],
                         'min_samples_split': [2, 4, 8],
                         'n_estimators': [5, 10, 15, 20]},
             return_train_score=True, scoring='neg_mean_absolute_error')


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [2, 4, 8, 15],
                         'min_samples_split': [2, 4, 8],
                         'n_estimators': [5, 10, 15, 20]},
             return_train_score=True, scoring='neg_mean_absolute_error')

In [24]:
# Print out the MAE score from the best-performing square
best_score = grid_rf_class.best_score_
print("Best score: ", best_score)

# Create a variable from the row related to the best-performing square
print("Best Index: ", grid_rf_class.best_index_)

print("Best Estimator: ", grid_rf_class.best_estimator_)

cv_results_df = pd.DataFrame(grid_rf_class.cv_results_)
best_row = cv_results_df.loc[[grid_rf_class.best_index_]]
print(best_row)

Best score:  -1.9635011523034003
Best Index:  46
Best Estimator:  RandomForestRegressor(max_depth=15, min_samples_split=8, n_estimators=15)
    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
46     136.924881      3.245486         1.139847        0.034768   

   param_max_depth param_min_samples_split param_n_estimators  \
46              15                       8                 15   

                                               params  split0_test_score  \
46  {'max_depth': 15, 'min_samples_split': 8, 'n_e...          -1.581096   

    split1_test_score  ...  mean_test_score  std_test_score  rank_test_score  \
46          -2.551279  ...        -1.963501        0.391884                1   

    split0_train_score  split1_train_score  split2_train_score  \
46           -1.796187           -1.401046           -1.670976   

    split3_train_score  split4_train_score  mean_train_score  std_train_score  
46            -1.80198           -1.906089         -1.715256     

In [ ]:
print(grid_rf_class.best_estimator_)

NameError: name 'grid_rf_class' is not defined